In [9]:
from os import listdir
from os.path import isfile, join
import argparse
#import cv2
import numpy as np
import sys
import os
import shutil
import random 
import math
import time
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt

In [16]:
class YOLO_Kmeans:

    def __init__(self, cluster_number, filename):
        self.cluster_number = cluster_number
        self.filename = filename

    def iou(self, boxes, clusters):  # 1 box -> k clusters
        n = boxes.shape[0]
        k = self.cluster_number

        box_area = boxes[:, 0] * boxes[:, 1]
        box_area = box_area.repeat(k)
        box_area = np.reshape(box_area, (n, k))

        cluster_area = clusters[:, 0] * clusters[:, 1]
        cluster_area = np.tile(cluster_area, [1, n])
        cluster_area = np.reshape(cluster_area, (n, k))

        box_w_matrix = np.reshape(boxes[:, 0].repeat(k), (n, k))
        cluster_w_matrix = np.reshape(np.tile(clusters[:, 0], (1, n)), (n, k))
        min_w_matrix = np.minimum(cluster_w_matrix, box_w_matrix)

        box_h_matrix = np.reshape(boxes[:, 1].repeat(k), (n, k))
        cluster_h_matrix = np.reshape(np.tile(clusters[:, 1], (1, n)), (n, k))
        min_h_matrix = np.minimum(cluster_h_matrix, box_h_matrix)
        inter_area = np.multiply(min_w_matrix, min_h_matrix)

        result = inter_area / (box_area + cluster_area - inter_area)
        return result

    def avg_iou(self, boxes, clusters):
        accuracy = np.mean([np.max(self.iou(boxes, clusters), axis=1)])
        return accuracy

    def kmeans(self, boxes, k, dist=np.median):
        box_number = boxes.shape[0]
        distances = np.empty((box_number, k))
        last_nearest = np.zeros((box_number,))
        np.random.seed()
        clusters = boxes[np.random.choice(
            box_number, k, replace=False)]  # init k clusters
        while True:

            distances = 1 - self.iou(boxes, clusters)

            current_nearest = np.argmin(distances, axis=1)
            if (last_nearest == current_nearest).all():
                break  # clusters won't change
            for cluster in range(k):
                clusters[cluster] = dist(  # update clusters
                    boxes[current_nearest == cluster], axis=0)

            last_nearest = current_nearest

        return clusters

    def result2txt(self, data):
        f = open("yolo_anchors.txt", 'w')
        row = np.shape(data)[0]
        for i in range(row):
            if i == 0:
                x_y = "%d,%d" % (data[i][0], data[i][1])
            else:
                x_y = ", %d,%d" % (data[i][0], data[i][1])
            f.write(x_y)
        f.close()

    def txt2boxes(self):
        f = open(self.filename, 'r')
        dataSet = []
        for line in f:
            infos = line.split(" ")
            length = len(infos)
            for i in range(1, length):
                width = int(infos[i].split(",")[2]) - \
                    int(infos[i].split(",")[0])
                height = int(infos[i].split(",")[3]) - \
                    int(infos[i].split(",")[1])
                dataSet.append([width, height])
        result = np.array(dataSet)
        f.close()
        return result

    def txt2clusters(self, boxes):
        # all_boxes = self.txt2boxes()
        all_boxes = boxes
        result = self.kmeans(all_boxes, k=self.cluster_number)
        result = result[np.lexsort(result.T[0, None])]
        self.result2txt(result)
        print("K anchors:\n {}".format(result))
        print("Accuracy: {:.2f}%".format(
            self.avg_iou(all_boxes, result) * 100))

In [10]:
detrac_anno_folder = './DETRAC-Train-Annotations-XML-v3'
anno_xmls = []
folder_name = []
xml_list = os.listdir(detrac_anno_folder)
for anno_xml in xml_list:
    anno_xmls.append(os.path.join(detrac_anno_folder, anno_xml))
    folder_name.append(anno_xml[:-7])
anno_xmls

['./DETRAC-Train-Annotations-XML-v3\\MVI_20011_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20012_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20032_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20033_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20034_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20035_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20051_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20052_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20061_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20062_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20063_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20064_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_20065_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_39761_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_39771_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_39781_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_39801_v3.xml',
 './DETRAC-Train-Annotations-XML-v3\\MVI_39811_v

In [18]:
%%time
out_cols = ['class', 'identity', 'x_center', 'y_center', 'width', 'height']

annotations_dim = []

for idx in range(len(anno_xmls)):
    print(f'Processing: {anno_xmls[idx]}')
    root = ET.parse(anno_xmls[idx]).getroot()
    for frame in root:
        if (frame.tag != 'frame'):
            continue
        for targets in frame[0]:
            width = round(float(targets[0].attrib['width']),2)
            height = round(float(targets[0].attrib['height']),2)
            # annotations_dim.append(tuple(map(float, (width, height))))
            annotations_dim.append([width, height])
annotations_dim = np.array(annotations_dim)
annotations_dim.shape

Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20011_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20012_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20032_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20033_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20034_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20035_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20051_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20052_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20061_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20062_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20063_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20064_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_20065_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_39761_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_39771_v3.xml
Processing: ./DETRAC-Train-Annotations-XML-v3\MVI_39781

(598281, 2)

In [21]:
annotations_dim[10000]

array([33.45, 27.48])

In [19]:
%%time
filename = 'anchor_exp_out.txt'
kmeans = YOLO_Kmeans(num_cluster, filename)
kmeans.txt2clusters(annotations_dim)

K anchors:
 [[ 24.47  19.35]
 [ 31.32  26.82]
 [ 41.75  31.93]
 [ 49.    42.81]
 [ 63.94  53.91]
 [ 67.35  37.27]
 [ 80.15  73.53]
 [ 96.15  49.  ]
 [110.65  99.13]
 [133.27  63.94]
 [173.8  107.66]
 [244.84 175.29]]
Accuracy: 80.29%
Wall time: 14.3 s
